In [3]:
import gc
import glob
import hashlib
import itertools
import json
import os
import random
import re
import subprocess
from collections import Counter
from os.path import join as pjoin
from tqdm import tqdm
import torch
from multiprocess import Pool

from others.logging import logger
from others.tokenization import BertTokenizer
from pytorch_transformers import XLNetTokenizer
import torch
import torch.nn as nn
from others.utils import clean
from prepro.utils import _get_word_ngrams
from stanfordcorenlp import StanfordCoreNLP
import xml.etree.ElementTree as ET
from prepro.data_builder import BertTokenizer
import nltk
nyt_remove_words = ["photo", "graph", "chart", "map", "table", "drawing"]
"""
conda activate py3.6
source /etc/profile
export CLASSPATH=/tf/project/stanford-corenlp-4.2.2/stanford-corenlp-4.2.2.jar 
"""

def recover_from_corenlp(s):
    s = re.sub(r' \'{\w}', '\'\g<1>', s)
    s = re.sub(r'\'\' {\w}', '\'\'\g<1>', s)




ModuleNotFoundError: No module named 'pytorch_transformers'

In [4]:
def load_json(p, lower):
    source = []
    tgt = []
    flag = False
    for sent in json.load(open(p))['sentences']:
        tokens = [t['word'] for t in sent['tokens']]
        if (lower):
            tokens = [t.lower() for t in tokens]
        if (tokens[0] == '@highlight'):
            flag = True
            tgt.append([])
            continue
        if (flag):
            tgt[-1].extend(tokens)
        else:
            source.append(tokens)

    source = [clean(' '.join(sent)).split() for sent in source]
    tgt = [clean(' '.join(sent)).split() for sent in tgt]
    return source, tgt

In [5]:
s,t = load_json(p='/tf/project/PreSumm-master/data/msmo_token/abb5721bac73e33bc733e3539520589e30832680.story.json', lower=False)

NameError: name 'clean' is not defined

In [ ]:
class Bert(nn.Module):
    def __init__(self, large, temp_dir, finetune=False):
        super(Bert, self).__init__()
        if(large):
            self.model = BertModel.from_pretrained('bert-large-uncased', cache_dir=temp_dir)
        else:
            self.model = BertModel.from_pretrained('bert-base-uncased', cache_dir=temp_dir)
        
        self.finetune = finetune

    def forward(self, x, segs, mask):
        if(self.finetune):
            top_vec, _ = self.model(x, segs, attention_mask=mask)
        else:
            self.eval()
            with torch.no_grad():
                top_vec, _ = self.model(x, segs, attention_mask=mask)
        return top_vec

In [120]:
bert_model = Bert(False,'../temp', True).cuda()

In [122]:
from pytorch_transformers import BertModel, BertConfig
class get_bert_input():
    def __init__(self,use_bert_basic_tokenizer=True,
                 max_src_ntokens_per_sent=200,
                 min_src_ntokens_per_sent=5,
                 max_src_nsents=100,
                 min_src_nsents=3,
                max_tgt_ntokens=500,
                min_tgt_ntokens=5,
                lower=True,
                bert_model=None):
      
        self.use_bert_basic_tokenizer=use_bert_basic_tokenizer
        self.max_src_ntokens_per_sent=max_src_ntokens_per_sent
        self.min_src_ntokens_per_sent=min_src_ntokens_per_sent
        self.max_src_nsents=max_src_nsents
        self.min_src_nsents=min_src_nsents
        self.max_tgt_ntokens=max_tgt_ntokens
        self.min_tgt_ntokens=min_tgt_ntokens
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        self.sep_token = '[SEP]'
        self.cls_token = '[CLS]'
        self.pad_token = '[PAD]'
        self.tgt_bos = '[unused0]'
        self.tgt_eos = '[unused1]'
        self.tgt_sent_split = '[unused2]'
        self.sep_vid = self.tokenizer.vocab[self.sep_token]
        self.cls_vid = self.tokenizer.vocab[self.cls_token]
        self.pad_vid = self.tokenizer.vocab[self.pad_token]
        self.bert = bert_model
    def tokenized_sentence(self,sentence):
        
        return nltk.word_tokenize(sentence)
    def process_sentence_bert(self,sentence):
        sentence_token = [self.tokenized_sentence(sentence)[:self.max_src_ntokens_per_sent]]
        sentence = [' '.join(sent) for sent in sentence_token]
        text = ' {} {} '.format(self.sep_token, self.cls_token).join(sentence)
        
        src_subtokens = self.tokenizer.tokenize(text)
        print( src_subtokens)
        src_subtokens = [self.cls_token] + src_subtokens + [self.sep_token]
        print( src_subtokens)
        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        return self.get_bert_vector(src_subtoken_idxs,[1]*len(src_subtoken_idxs),[True]*len(src_subtoken_idxs))
    def get_bert_vector(self,src,seq,mask):
        src = torch.Tensor(src).cuda().unsqueeze(0)
        seq = torch.Tensor(seq).cuda().unsqueeze(0)
        mask =  torch.Tensor(mask).cuda().unsqueeze(0)
        return self.bert(src.long(), seq.long(), mask)

In [123]:
x = get_bert_input(bert_model=bert_model)

In [124]:
print(x.process_sentence_bert('good good study, day day up.'))

['good', 'good', 'study', ',', 'day', 'day', 'up', '.']
['[CLS]', 'good', 'good', 'study', ',', 'day', 'day', 'up', '.', '[SEP]']
tensor([[[-0.0231,  0.0306, -0.1991,  ..., -0.1348,  0.4759,  0.3542],
         [-0.0090,  0.2746,  0.1730,  ..., -0.6673, -0.5767, -0.7292],
         [ 0.0872,  0.1668,  0.0089,  ..., -0.3547, -0.3139, -0.8581],
         ...,
         [ 0.0624,  0.0135,  0.0500,  ...,  0.6453,  0.1331, -0.1702],
         [ 0.8105,  0.0055, -0.3933,  ...,  0.1931, -0.4223, -0.4322],
         [ 1.0457, -0.0533, -0.3945,  ...,  0.1900, -0.4931, -0.3089]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward>)


In [2]:
from models import model_builder

ModuleNotFoundError: No module named 'pytorch_transformers'

In [1]:
ext_model = model_builder.ExtSummarizer()

NameError: name 'model_builder' is not defined